**Download pre-trained model**

In [ ]:
import torch
from torchvision.models import resnet50
from torchvision.models import ResNet50_Weights

# Load default pre-trained weights
weights = ResNet50_Weights.DEFAULT
resnet_model = resnet50(weights=weights)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.is_available()

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 110MB/s]


False

split the data into training, validation and testing while making sure that every category contains the 10 classes

In [ ]:
!pip install tqdm #Shows a nice progress bar during the copying process

In [ ]:
#import os
#import shutil
#from sklearn.model_selection import train_test_split
#from torchvision.datasets import ImageFolder
#import numpy as np
#from tqdm import tqdm

# Paths
#source_dir = "/content/drive/MyDrive/EuroSAT"     # Folder containing 10 class folders
#output_base = "/content/drive/MyDrive/EuroSAT_splitted"    # New folder to store train/val/test

# Load dataset
#dataset = ImageFolder(source_dir)
#image_paths = [s[0] for s in dataset.samples]
#labels = [s[1] for s in dataset.samples]
#class_names = dataset.classes

# Split
#train_paths, temp_paths, train_labels, temp_labels = train_test_split(
#    image_paths, labels, test_size=0.3, stratify=labels, random_state=42)

#val_paths, test_paths, val_labels, test_labels = train_test_split(
#    temp_paths, temp_labels, test_size=0.5, stratify=temp_labels, random_state=42)

# Copy function
#def copy_files(paths, labels, dest):
#    for path, label in tqdm(zip(paths, labels), total=len(paths)):
#        class_folder = class_names[label]
#        target_dir = os.path.join(dest, class_folder)
#       os.makedirs(target_dir, exist_ok=True)
#       shutil.copy(path, os.path.join(target_dir, os.path.basename(path)))

# Copy to Drive
#print("Copying training data...")
#copy_files(train_paths, train_labels, os.path.join(output_base, "train"))

#print("Copying validation data...")
#copy_files(val_paths, val_labels, os.path.join(output_base, "val"))

#print("Copying test data...")
#copy_files(test_paths, test_labels, os.path.join(output_base, "test"))


load EuroSAT_splitted into loacal colab

In [ ]:
!cp -r /content/drive/MyDrive/EuroSAT_splitted /content/

**Freeze the base layers**

In [ ]:
#resnet_model.requires_grad_(False)
for param in resnet_model.parameters():
    param.requires_grad = False

**Adding layer**

In [ ]:
import torch.nn as nn
N_CLASSES = 10
#my_model = nn.Sequential(
#    resnet_model,
#    nn.Linear(1000, N_CLASSES)
#)

## Instead of adding a new layer after resnet_model, replace its .fc:
resnet_model.fc = nn.Sequential(
    nn.Dropout(p=0.5),
    nn.Linear(resnet_model.fc.in_features, 256),
    nn.ReLU(),
    nn.Dropout(p=0.3),
    nn.Linear(256, 10)
)
my_model = resnet_model










**Loss function**

In [ ]:
loss_function = nn.CrossEntropyLoss()

**Optimizer**

In [ ]:
from torch.optim import Adam
optimizer = Adam(my_model.parameters())

**Model** **compilation**

In [ ]:
my_model = torch.compile(my_model.to(device), backend="eager")

**Pre_processing transforms**

In [ ]:
import torchvision.transforms.v2 as transforms
pre_trans = weights.transforms()

**Custom Dataset**

In [ ]:
import os
import glob
import torch
from torch.utils.data import Dataset
from PIL import Image

DATA_LABELS = ["AnnualCrop", "Forest", "HerbaceousVegetation", "Highway", "Industrial",
               "Pasture", "PermanentCrop", "Residential", "River", "SeaLake"]

class MyDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.image_paths = []
        self.labels = []
        self.transform = transform

        for l_idx, label in enumerate(DATA_LABELS):
            paths = glob.glob(os.path.join(data_dir, label, '*.jpg'))
            for path in paths:
                self.image_paths.append(path)
                self.labels.append(l_idx)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert("RGB")
        if self.transform:
            image = self.transform(image)
        label = torch.tensor(self.labels[idx]).long()
        return image, label

    def __len__(self):
        return len(self.image_paths)


**Data** **augmentation**

In [ ]:
from torchvision.transforms.v2 import (
    Compose, RandomResizedCrop, ColorJitter, RandomHorizontalFlip, RandomRotation, ToTensor, Normalize
)

random_trans = Compose([
    RandomResizedCrop(64, scale=(0.7, 1.0)),
    ColorJitter(brightness=0.3, contrast=0.3, saturation=0.2, hue=0.02),
    RandomHorizontalFlip(),
    RandomRotation(20),
    ToTensor(),
    Normalize(mean=[0.485, 0.456, 0.406],
              std=[0.229, 0.224, 0.225])
])

**Preprocessing**

In [ ]:
import torch
import os
from torch.utils.data import DataLoader
from random import shuffle

n = 32

#train_path = "/content/drive/MyDrive/EuroSAT_splitted/train"
train_path = "/content/EuroSAT_splitted/train"
train_data = MyDataset(train_path, transform = random_trans)
train_loader = DataLoader(train_data, batch_size = n, shuffle = True)
train_N = len(train_loader.dataset)

#valid_path = "/content/drive/MyDrive/EuroSAT_splitted/val"
valid_path = "/content/EuroSAT_splitted/val"
valid_data = MyDataset(valid_path, transform = random_trans)
valid_loader = DataLoader(valid_data, batch_size = n)
valid_N = len(valid_loader.dataset)

**Train function**

In [ ]:
def train(model, train_loader, train_N, random_trans, optimizer, loss_function):
    loss = 0
    accuracy = 0

    model.train()
    for x, y in train_loader:
        output = model(x.to(device))
        optimizer.zero_grad()
        batch_loss = loss_function(output, y.to(device))
        batch_loss.backward()
        optimizer.step()

        loss += batch_loss.item()
        accuracy += get_batch_accuracy(output, y.to(device), train_N)
    print('Train - Loss: {:.4f} Accuracy: {:.4f}'.format(loss, accuracy))

**Validate function**

In [ ]:
def validate(model, valid_loader, valid_N, loss_function):
    loss = 0
    accuracy = 0

    model.eval()
    with torch.no_grad():
        for x, y in valid_loader:
            output = model(x.to(device))

            loss += loss_function(output, y.to(device)).item()
            accuracy += get_batch_accuracy(output, y.to(device), valid_N)
    print('Valid - Loss: {:.4f} Accuracy: {:.4f}'.format(loss, accuracy))

**Accuracy function**

In [ ]:
def get_batch_accuracy(output, y, N):
    pred = output.argmax(dim=1, keepdim=True)
    correct = pred.eq(y.view_as(pred)).sum().item()
    return correct / N

**Training loop**

In [ ]:
epochs = 10

for epoch in range(epochs):
    print('Epoch: {}'.format(epoch))
    train(my_model, train_loader, train_N, random_trans, optimizer, loss_function)
    validate(my_model, valid_loader, valid_N, loss_function)

Epoch: 0
Train - Loss: 721.6732 Accuracy: 0.5978
Valid - Loss: 97.3316 Accuracy: 0.7496
Epoch: 1
Train - Loss: 607.0313 Accuracy: 0.6531
Valid - Loss: 99.7766 Accuracy: 0.7451
Epoch: 2
Train - Loss: 579.3428 Accuracy: 0.6744
Valid - Loss: 91.6413 Accuracy: 0.7646
Epoch: 3
Train - Loss: 578.5622 Accuracy: 0.6710
Valid - Loss: 87.1741 Accuracy: 0.7745
Epoch: 4
Train - Loss: 572.3662 Accuracy: 0.6738
Valid - Loss: 89.0651 Accuracy: 0.7686
Epoch: 5
Train - Loss: 565.6212 Accuracy: 0.6830
Valid - Loss: 90.3636 Accuracy: 0.7609
Epoch: 6
Train - Loss: 556.6621 Accuracy: 0.6818
Valid - Loss: 88.4901 Accuracy: 0.7792
Epoch: 7
Train - Loss: 556.3090 Accuracy: 0.6811
Valid - Loss: 81.4402 Accuracy: 0.7957
Epoch: 8
Train - Loss: 551.7430 Accuracy: 0.6841
Valid - Loss: 88.0160 Accuracy: 0.7839
Epoch: 9
Train - Loss: 550.3698 Accuracy: 0.6853
Valid - Loss: 90.2577 Accuracy: 0.7604


**Unfreeze the model**

In [ ]:
resnet_model.requires_grad_(True)
optimizer = Adam(my_model.parameters(), lr=.00010)

**Data augmentation for the final training** (Resnet expects 224*224 images)

In [ ]:
random_trans224 = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.Resize((224, 224)),  # <-- full size for ResNet
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

/usr/local/lib/python3.11/dist-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


**Preprocessing**

In [ ]:
import torch
import os
from torch.utils.data import DataLoader
from random import shuffle

n = 32

#train_path = "/content/drive/MyDrive/EuroSAT_splitted/train"
train_path = "/content/EuroSAT_splitted/train"
train_data = MyDataset(train_path, transform = random_trans224)
train_loader = DataLoader(train_data, batch_size = n, shuffle = True)
train_N = len(train_loader.dataset)

#valid_path = "/content/drive/MyDrive/EuroSAT_splitted/val"
valid_path = "/content/EuroSAT_splitted/val"
valid_data = MyDataset(valid_path, transform = random_trans224)
valid_loader = DataLoader(valid_data, batch_size = n)
valid_N = len(valid_loader.dataset)

**Training loop**

In [ ]:
epochs = 2

for epoch in range(epochs):
    print('Epoch: {}'.format(epoch))
    train(my_model, train_loader, train_N, random_trans224, optimizer, loss_function)
    validate(my_model, valid_loader, valid_N, loss_function)

Epoch: 0
Train - Loss: 157.0211 Accuracy: 0.9119
Valid - Loss: 12.9785 Accuracy: 0.9615
Epoch: 1
Train - Loss: 73.6918 Accuracy: 0.9583
Valid - Loss: 8.9507 Accuracy: 0.9778


**Save the model**

In [ ]:
import os

# Ensure the directory exists
os.makedirs("/content/drive/MyDrive/checkpoints", exist_ok=True)

# Now save
torch.save(my_model.state_dict(), f"/content/drive/MyDrive/checkpoints/model_epoch{epoch}.pth")
